<a href="https://colab.research.google.com/github/RicoStaedeli/NLP2025_CQG/blob/main/4_Finetuned_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetuned Predictions
In this file we generate the finetuned predictions

## Setup

In [1]:
!pip install -U transformers
!pip install accelerate bitsandbytes
!pip install -U peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [2]:
import torch
from google.colab import userdata
import logging
import transformers
import os
import gc
from transformers import AutoModelForCausalLM, AutoTokenizer
from bitsandbytes import nn as bnb_nn
from tqdm.auto import tqdm
import pandas as pd
import json
from collections import defaultdict
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [3]:
token = userdata.get('GITHUB')
repo_url = f"https://{token}@github.com/RicoStaedeli/NLP2025_CQG.git"

!git clone {repo_url}

Cloning into 'NLP2025_CQG'...
remote: Enumerating objects: 1000, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 1000 (delta 73), reused 54 (delta 54), pack-reused 913 (from 1)
Receiving objects: 100% (1000/1000), 46.08 MiB | 20.70 MiB/s, done.
Resolving deltas: 100% (549/549), done.
Updating files: 100% (95/95), done.


In [4]:
os.chdir("NLP2025_CQG")
!ls

1_Information_preprocessing.md	      Doc
1_Preprocessing.ipynb		      Evaluation
2_Baseline_Generation.ipynb	      INFORMATION.md
2_Information_Baseline_Generation.md  LICENSE
3_Evaluation.ipynb		      Logs
4_Finetuned_Generation.ipynb	      README.md
5_Evaluation_Analytics.ipynb	      requirements.txt
Data				      Training
Development			      Utils


In [5]:
################################################################################
#######################   PATH VARIABLES        ################################
################################################################################

BASE_MODEL = "unsloth/Meta-Llama-3.1-8B-Instruct"
model_name= "Meta-Llama-3.1-1B-Instruct_SFT_2"
model_id= f"ricostaedeli/{model_name}"

test_dataset_path = f"Data/Processed/test.csv"

results_path = os.path.join(os.getcwd(), f"Evaluation/Results/results_{model_name}.json")

log_base_path = f"Logs/"
os.makedirs(log_base_path, exist_ok=True)

log_path = log_base_path + "4_cqs_generation_SFT_2.log"


################################################################################
#######################   STATIC VARIABLES      ################################
################################################################################


In [6]:
# Setup logger manually
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Create file handler (only if not already added)
if not logger.handlers:
    fh = logging.FileHandler(log_path)
    fh.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    fh.setFormatter(formatter)
    logger.addHandler(fh)

# Detect device
device = torch.device(
    "mps" if torch.backends.mps.is_available()
    else "cuda" if torch.cuda.is_available()
    else "cpu"
)

# Log the device info
logger.info("--------  Start with Baseline Generation  -------------")
logger.info(f'Device selected: {device}')
logger.info(f'Results Path: {results_path}')
logger.info(f'Log Path: {log_path}')
logger.info("--------------------------------------------------------")

INFO:__main__:--------  Start with Baseline Generation  -------------
INFO:__main__:Device selected: cuda
INFO:__main__:Results Path: /content/NLP2025_CQG/Evaluation/Results/results_Meta-Llama-3.1-1B-Instruct_SFT_2-lora.json
INFO:__main__:Log Path: Logs/4_cqs_generation_SFT_2.log
INFO:__main__:--------------------------------------------------------


## Generate Answers

Do this cell only when you load the complete model

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="cuda",
    quantization_config={
        "load_in_4bit": True,
        "bnb_4bit_compute_dtype": torch.float16,
    },
    low_cpu_mem_usage=True,
)

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/924 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

In [9]:
def generate_response(prompt_texts, max_new_tokens=64):
    inputs = tokenizer(prompt_texts, return_tensors="pt", padding=True, truncation=True).to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        eos_token_id=tokenizer.eos_token_id,
    )

    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    responses = []
    for full_output in decoded:
        print(full_output)
        # Extract text after ### Response:
        if "### Response:" in full_output:
            response = full_output.split("### Response:")[-1].strip()
        else:
            response = full_output.strip()

        # Remove leading 'assistant\n\n' if present
        if response.lower().startswith("assistant"):
            response = response[len("assistant"):].lstrip("\n ").strip()

        responses.append(response)
    return responses

In [10]:
# del model
# gc.collect()
# torch.cuda.empty_cache()

In [10]:
schemas = {
"CauseToEffect": """'Cause to Effect' with the examples:
How strong is the generalisation that if <eventA> then <eventB>?
Are there other factors in this particular case that could have interfered with the event of‘<eventB>’?""",

"ExpertOpinion": """'Expert Opinion' with the examples:
Is <expertE> a genuine expert in <domainD>?
Is <eventA> consistent with what other experts in <domainD> say? """,

"Analogy": """'Analogy' with the examples:
Are <C1> and <C2> similar in the respect cited?
Is <eventA> true in <C1>?""",

"FearAppeal": """'Fear Appeal' with the examples:
Is <eventB> bad? Why and to whom is it bad?
Is <eventA> away to prevent <eventB>?"""
}

In [11]:
results = {}
chunk_size = 20

for chunk in pd.read_csv(test_dataset_path, chunksize=chunk_size):
    contexts = chunk['input'].tolist()
    ids = chunk['id'].tolist()
    prompts = []
    input_ids = []
    schema_ids = []


    for idx, input_text in enumerate(contexts):
        input_id = ids[idx]
        for schema_name, schema_template in schemas.items():
            messages = [
                {"role": "system", "content": "You generate concise, critical, single-sentence questions for argumentative contexts, matching specified question schemas."},
                {"role": "user", "content": f"""### Instruction:
Generate one critical question addressing the provided context. Ensure it matches the schema:
{schema_template}

### Context:
{input_text}

Respond with only the generated question.

### Response:

"""}
            ]
            prompt_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            prompts.append(prompt_text)
            input_ids.append(input_id)
            schema_ids.append(schema_name)

    batch_size = 64
    num_batches = (len(prompts) + batch_size - 1) // batch_size

    with tqdm(total=num_batches, desc="Generating Critical Questions", leave=True) as pbar:
        for batch_start in range(0, len(prompts), batch_size):
            batch_prompts = prompts[batch_start:batch_start+batch_size]
            batch_outputs = generate_response(batch_prompts, max_new_tokens=256)

            for curr_id, schema_name, output in zip(
                input_ids[batch_start:batch_start + batch_size],
                schema_ids[batch_start:batch_start + batch_size],
                batch_outputs
                ):
                if curr_id not in results:
                    results[curr_id] = {
                        "input": chunk.loc[chunk['id'] == curr_id, 'input'].values[0],
                        "cqs": []
                    }
                results[curr_id]['cqs'].append({
                    "schema": schema_name,
                    "cq": output
                    })

            torch.cuda.empty_cache()
            gc.collect()
            pbar.update(1)

with open(results_path, 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"Results saved to {results_path}")


Generating Critical Questions:   0%|          | 0/2 [00:00<?, ?it/s]

system

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You generate concise, critical, single-sentence questions for argumentative contexts, matching specified question schemas.user

### Instruction:
Generate one critical question addressing the provided context. Ensure it matches the schema:
'Cause to Effect' with the examples:
How strong is the generalisation that if <eventA> then <eventB>?
Are there other factors in this particular case that could have interfered with the event of‘<eventB>’?

### Context:
CLINTON: "which may prove to be an intelligence benefit
we've got to do everything we can to vacuum up intelligence from Europe, from the Middle East
That means we've got to work more closely with our allies, and that's something that Donald has been very dismissive of
We're working with NATO, the longest military alliance in the history of the world, to really turn our attention to terrorism
We're working with our friends in the Middle East, many of which, as you k

Generating Critical Questions:   0%|          | 0/2 [00:00<?, ?it/s]

system

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You generate concise, critical, single-sentence questions for argumentative contexts, matching specified question schemas.user

### Instruction:
Generate one critical question addressing the provided context. Ensure it matches the schema:
'Cause to Effect' with the examples:
How strong is the generalisation that if <eventA> then <eventB>?
Are there other factors in this particular case that could have interfered with the event of‘<eventB>’?

### Context:
CL: "Well, both sides of the transaction suffer as a result of ignoring moral hazard
because moral hazard is a natural part of the process of lending and borrowing
And if you suppress it, then in fact you're in a situation where the wealth that you look like you're generating is fool’s gold, not real wealth at all."

Respond with only the generated question.

### Response:assistant

### Response:
system

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024


Generating Critical Questions:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Commit & Push

In [ ]:
!git config --global user.name "Rico Städeli"
!git config --global user.email "rico@yabrriga.ch"


commit_message = f"Finetuned generation"
!git add .
!git commit -m "{commit_message}"
!git push

[main dcd4e0e] Finetuned generation
 2 files changed, 3913 insertions(+), 3908 deletions(-)
 rewrite Evaluation/Results/results_Meta-Llama-3.1-1B-Instruct_SFT_2.json (73%)
Enumerating objects: 13, done.
Counting objects: 100% (13/13), done.
Delta compression using up to 12 threads
Compressing objects: 100% (7/7), done.
Writing objects: 100% (7/7), 5.99 KiB | 558.00 KiB/s, done.
Total 7 (delta 4), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (4/4), completed with 4 local objects.
To https://github.com/RicoStaedeli/NLP2025_CQG.git
   d919344..dcd4e0e  main -> main
